In [ ]:

# Session 24: Assignment 1

In [ ]:
"""
Problem Statement

Predicting Survival in the Titanic Data Set
We will be using a decision tree to make predictions about the Titanic data set from
Kaggle. This data set provides information on the Titanic passengers and can be used to
predict whether a passenger survived or not.

Loading Data and modules

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn
from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics

from sklearn.metrics import classification_report

Url=https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv
titanic = pd.read_csv(url)
titanic.columns =['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']

You use only Pclass, Sex, Age, SibSp (Siblings aboard), Parch (Parents/children aboard),and Fare to predict whether a passenger survived.

"""

In [3]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

In [4]:
titanic=pd.read_csv("https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv")
titanic.head(7)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S


In [5]:
print(titanic.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [6]:
titanic['Sex'],_=pd.factorize(titanic['Sex'])
titanic.head(6)
# male=0,Female=1


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",0,NaN,0,0,330877,8.4583,NaN,Q


In [7]:
data=titanic[['Pclass','Sex','Age','SibSp','Parch','Fare','Survived']]
print(data.head(7))
print(data.shape)


   Pclass  Sex   Age  SibSp  Parch     Fare  Survived
0       3    0  22.0      1      0   7.2500         0
1       1    1  38.0      1      0  71.2833         1
2       3    1  26.0      0      0   7.9250         1
3       1    1  35.0      1      0  53.1000         1
4       3    0  35.0      0      0   8.0500         0
5       3    0   NaN      0      0   8.4583         0
6       1    0  54.0      0      0  51.8625         0
(891, 7)


In [ ]:
# Taking care of NaN values in Age column.
# We can either impute, use mean or median or any random number between mean-std, mean+std to fill
# Age missing values. 
# or we can dropna as it is only 19% of the total instances.
# Can use to fill by mean(if no outliers, or median(if outliers)>>>dataframe.Column_Name.fillna(dataframe.Column_Name.mean(),inplace=True)

In [8]:
# Will drop all rows that have any missing values.
data.dropna(inplace=True)
data.head(10)

C:\Users\gaura\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,3,0,22.0,1,0,7.2500,0
1,1,1,38.0,1,0,71.2833,1
2,3,1,26.0,0,0,7.9250,1
3,1,1,35.0,1,0,53.1000,1
4,3,0,35.0,0,0,8.0500,0
6,1,0,54.0,0,0,51.8625,0
7,3,0,2.0,3,1,21.0750,0
8,3,1,27.0,0,2,11.1333,1
9,2,1,14.0,1,0,30.0708,1
10,3,1,4.0,1,1,16.7000,1


In [9]:
data.shape

(714, 7)

In [11]:
X=data.drop(['Survived'],axis=1)
y=data['Survived']
print(X.head(10))
print(X.shape)
print(y.shape)

    Pclass  Sex   Age  SibSp  Parch     Fare
0        3    0  22.0      1      0   7.2500
1        1    1  38.0      1      0  71.2833
2        3    1  26.0      0      0   7.9250
3        1    1  35.0      1      0  53.1000
4        3    0  35.0      0      0   8.0500
6        1    0  54.0      0      0  51.8625
7        3    0   2.0      3      1  21.0750
8        3    1  27.0      0      2  11.1333
9        2    1  14.0      1      0  30.0708
10       3    1   4.0      1      1  16.7000
(714, 6)
(714,)


In [17]:
# Now data is clean
# Train test split
# splitting data: training = 75% and test=25%

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.25,random_state=2)

In [18]:
# Import,Instantiate and Fit
from sklearn import tree
dtree= tree.DecisionTreeClassifier(criterion='entropy',max_depth=3,random_state=8)
# criterion can be entropy or gini etc

In [19]:
dtree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=8,
            splitter='best')

In [20]:
# use the model to make predictions with the test data
y_predict=dtree.predict(X_test)

In [21]:
# Evaluating the model's performance
misclassified=(y_test != y_predict).sum()
print('Misclassified :{}'.format(misclassified))

accuracy=metrics.accuracy_score(y_test,y_predict)
print("Accuracy: {}".format(accuracy))

Misclassified :29
Accuracy: 0.8379888268156425


In [ ]:
# Lets predict: A female travelling first class of 29 years of age with 1 sibling whoc has paid 80 dollars fare.
# Is she going to survive or not?

In [25]:
y_predict=dtree.predict([[1,1,29.0,1,0,80.0]])
print(y_predict)

# 0 is not survived and 1 means survived.

[1]


In [ ]:
# Above array[1] is the result which means she survives.

In [ ]:
# Below is a man of age 35 travelling third class who paid 12 dollars for the ticket.

In [26]:
y_predict=dtree.predict([[3,0,35.0,1,0,12.0]])
print(y_predict)

[0]


In [ ]:
# hard luck . he is not going to survive.